<a href="https://colab.research.google.com/github/naokishibuya/cuda-q-experiments/blob/main/notebooks/01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 — CUDA-Q Intro (Google Colab-ready)

This notebook makes sure it can run with **CUDA-Q** on Google Colab and verifies the install with a Bell state and a tiny variational example.

1. Check GPU in the Google Colab's Edit > Notebook settings
2. Install CUDA-Q (`pip install cudaq`)
3. Run a Bell state example and sample results
4. Evaluate an observable with `cudaq.observe` in a small 1-qubit variational loop


## Install CUDA-Q (Python API)

If not already installed, install CUDA-Q with pip:

In [ ]:
%pip install --upgrade pip
%pip install cudaq

In [2]:
import cudaq

print('CUDA-Q version:', getattr(cudaq, '__version__', 'unknown'))

CUDA-Q version: CUDA-Q Version 0.12.0 (https://github.com/NVIDIA/cuda-quantum 6adf92bcda4df7465e4fe82f1c8f782ae69d8bd2)


## Hello, Bell State
Prepare a 2-qubit Bell state and sample bitstrings. You should mainly see `00` and `11`.


In [3]:
# Create a Bell state and sample it
kernel = cudaq.make_kernel()
q = kernel.qalloc(2)

kernel.h(q[0])
kernel.cx(q[0], q[1])
kernel.mz(q)

result = cudaq.sample(kernel, shots_count=2000)
print(result)


{ 00:1022 11:978 }



## Tiny Variational Example with `observe`
We target an expectation value $\langle Z \rangle = 0.5$ using a single-parameter circuit $R_y(\theta)|0\rangle$.

Analytically, $\langle Z \rangle = \cos(\theta)$ for this ansatz, so the exact solution is $\theta = \arccos(0.5) = \pi/3 \approx 1.0472$.

Below we:
1. Build a parameterized kernel
2. Define an observable `Z`
3. Use `cudaq.observe` to evaluate $\langle Z \rangle$ without destructive measurement
4. Do a simple grid search to get close to the exact angle


In [4]:
from math import pi, acos

# 1. Build a parameterized kernel: one qubit, RY(theta)
kernel, theta = cudaq.make_kernel(float)
q = kernel.qalloc()
kernel.ry(theta, q)

# 2. Define an observable and target
H = cudaq.spin.z(0)
target = 0.5
theta_exact = acos(target)
print('Exact theta (acos(target)) =', theta_exact)

# 3. Define an objective function
#    Use cudaq.observe to evaluate <Z>(t) for given t
def objective(t: float) -> float:
    # Return |<Z>(t) - target| using cudaq.observe.
    obs = cudaq.observe(kernel, H, t)
    exp = obs.expectation()
    return abs(exp - target)

# 4. Do a simple grid search over theta in [0, pi]
print('\nScanning coarse grid...')
best_t, best_val = None, 1e9
for i in range(0, 101):
    t = i * pi / 100
    val = objective(t)
    if val < best_val:
        best_t, best_val = t, val
    if i % 25 == 0:
        print(f'i={i:3d}  t={t:.6f}  |<Z> - target| = {val:.6f}')

print(f"\nBest coarse theta ≈ {best_t:.6f}  objective ≈ {best_val:.6e}")
print('Delta from exact ≈', abs(best_t - theta_exact))


Exact theta (acos(target)) = 1.0471975511965979

Scanning coarse grid...
i=  0  t=0.000000  |<Z> - target| = 0.500000
i= 25  t=0.785398  |<Z> - target| = 0.207107
i= 50  t=1.570796  |<Z> - target| = 0.500000
i= 75  t=2.356194  |<Z> - target| = 1.207107
i=100  t=3.141593  |<Z> - target| = 1.500000

Best coarse theta ≈ 1.036726  objective ≈ 9.041518e-03
Delta from exact ≈ 0.010471975511965992
